In [1]:
#!/usr/bin/env python3
"""
DHN Performance Toolkit - Main Analysis Script
==============================================

District Heating Network performance analysis with:
- COP & Energy Analysis  
- Velocity Analysis
- Automated Reporting

Usage:
    1. Copy config_template.py to config_local.py
    2. Adjust paths in config_local.py
    3. Run: python main_analysis.py
"""

# Try to import local config, fallback to template
try:
    from config_local import SCENARIOS, DEFAULT_PARAMS
    print("✅ Using local configuration")
except ImportError:
    print("⚠️ config_local.py not found - using template")
    print("💡 Copy config_template.py to config_local.py and adjust paths")
    from config_template import SCENARIOS, DEFAULT_PARAMS



✅ Using local configuration


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# UESGraphs
from uesgraphs.uesgraph import UESGraph
import uesgraphs.analyze as analyze

In [3]:
from modules.decentral_pump_power import calculate_decentral_pump_energy

In [4]:
# Select scenario
scenario_key = "Scenario 1"  # Adjust this
scenario = SCENARIOS[scenario_key]

print(f"📋 Analyzing: {scenario['name']}")
print(f"📂 Output: {scenario['output_dir']}")

📋 Analyzing: Analysis 1
📂 Output: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\outputs


In [5]:
# Data Loading Functions
def get_dataframe(mask, file_path, uesgraph):
    """Load data for a specific mask pattern"""
    filter_list = []
    for node in uesgraph.nodelist_building:
        if not uesgraph.nodes[node]["is_supply_heating"]:
            name_bldg = uesgraph.nodes[node]["name"]
            filter_pattern = mask.format(name_bldg=name_bldg)
            filter_list.append(filter_pattern)
    
    df = analyze.process_simulation_result(file_path=file_path, filter_list=filter_list)
    df = analyze.prepare_DataFrame(
        df, 
        base_date=datetime.strptime(DEFAULT_PARAMS["start_date"], "%Y-%m-%d"), 
        end_date=datetime.strptime(DEFAULT_PARAMS["end_date"], "%Y-%m-%d"),
        time_interval=DEFAULT_PARAMS["time_interval"]
    )
    
    # Simplify column names
    import re
    pattern = re.compile(r'T([^.]+)')
    new_columns = []
    for col in df.columns:
        match = pattern.search(col)
        if match:
            new_columns.append(f"T{match.group(1)}")
        else:
            new_columns.append(col)
    df.columns = new_columns
    
    return df

# Load UESGraph
uesgraph = UESGraph()
uesgraph.from_json(path=scenario["json_path"], network_type="heating")
print(f"✅ Network loaded: {len(uesgraph.nodes)} nodes, {len(uesgraph.edges)} edges")

read nodes...
******
 input_ids were {'buildings': None, 'nodes': '5f4c976f-bea3-476b-875b-c4d08dfc2057', 'pipes': None, 'supplies': None}
...finished
✅ Network loaded: 68 nodes, 67 edges


In [6]:
# Load pump power data
PUMP_MASK = 'networkModel.demandT{name_bldg}.simplePumpConstFlow.pumpHeating.P$'
df_pump_power = get_dataframe(PUMP_MASK, scenario["data_path"], uesgraph)

print(f"📊 Pump data loaded: {df_pump_power.shape}")
print(f"📊 Buildings: {list(df_pump_power.columns)}")
# Run pump analysis using the module
pump_results = calculate_decentral_pump_energy(
    df_pump_power, 
    output_dir=scenario["output_dir"],
    store_result=False,
)


Processing: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\Sim20250612_154050_1\Results\Sim20250612_154050_1_inputs.gzip
📊 Pump data loaded: (35041, 32)
📊 Buildings: ['T56_wa2b_efh2', 'T54_wa2b_efh1', 'T48_wa2b_rhh3_2', 'T50_wa2b_rhh3_3', 'T44_wa2b_dhh6_1', 'T42_wa2b_dhh6_2', 'T58_wa2b_dhh3_1', 'T36_wa2b_dhh3_2', 'T38_wa2b_dhh4_1', 'T40_wa2b_dhh4_2', 'T42_wa2b_dhh2_1', 'T34_wa2b_dhh2_2', 'T32_wa2b_rhh2_1', 'T30_wa2b_rhh2_2', 'T28_wa2b_rhh2_3', 'T27_wa3_dhh1_1', 'T25_wa3_dhh1_2', 'T23_wa3_rhh1_2', 'T21_wa3_rhh1_3', 'T37_wa3_dhh3_1', 'T35_wa3_dhh3_2', 'T33_wa3_dhh2_1', 'T29_wa3_dhh2_2', 'T43_wa3_dhh5_1', 'T41_wa3_dhh5_2', 'T45_wa3_dhh6_1', 'T49_wa3_rhh2_2', 'T47_wa3_rhh2_3', 'T26_wa2b_rhh1_1', 'T24_wa2b_rhh1_2', 'T22_wa2b_rhh1_3', 'T18_wa2b_dhh1_1']


In [7]:
from modules.heat_pump_analyzer import *
import os

In [8]:
# HP Power & Energy Analysis

# Load HP and thermal data
HP_POWER_MASK = 'networkModel.demandT{name_bldg}.heatPumpFixDeltaT.heaPum.P$'
THERMAL_MASK = 'networkModel.demandT{name_bldg}.heatPumpFixDeltaT.heaPum.QCon_flow$'

df_hp_power = get_dataframe(HP_POWER_MASK, scenario["data_path"], uesgraph)
df_thermal = get_dataframe(THERMAL_MASK, scenario["data_path"], uesgraph)

print(f"📊 HP Power data: {df_hp_power.shape}")
print(f"📊 Thermal data: {df_thermal.shape}")


Processing: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\Sim20250612_154050_1\Results\Sim20250612_154050_1_inputs.gzip
Processing: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\Sim20250612_154050_1\Results\Sim20250612_154050_1_inputs.gzip
📊 HP Power data: (35041, 32)
📊 Thermal data: (35041, 32)


In [9]:

# Monthly energy analysis using your imported functions
monthly_energy, annual_stats = analyze_monthly_energy_cop(
    df_thermal_power=df_thermal,
    df_hp_electrical_power=df_hp_power, 
    df_pump_electrical_power=df_pump_power,  # From previous cell
    time_interval_hours=DEFAULT_PARAMS["time_interval_hours"],
    save_path=os.path.join(scenario["output_dir"], "monthly_energy_analysis.csv")
)


Converting power to energy using 0.25h intervals...

MONTHLY ENERGY-BASED COP ANALYSIS
Month      Thermal    Electrical   COP    COP(HP)  Activity
           [MWh]      [MWh]               only     [%]     
--------------------------------------------------------------------------------
January    36465.5    8239.3       4.43   4.53     51.6    
February   30856.1    7109.2       4.34   4.43     45.4    
March      25094.6    5940.2       4.22   4.31     31.5    
April      15862.2    3990.5       3.98   4.05     16.8    
May        10233.1    2871.3       3.56   3.80     6.9     
June       8670.1     2525.9       3.43   3.79     5.2     
July       8378.2     2447.4       3.42   3.85     4.9     
August     8170.8     2372.6       3.44   3.83     4.8     
September  8426.8     2294.0       3.67   3.85     5.5     
October    16450.4    3742.5       4.40   4.48     19.0    
November   28157.8    6176.6       4.56   4.67     41.7    
December   36484.4    8065.5       4.52   4.63     5

In [11]:
df_cop, stats = calculate_heat_pump_cop(
    df_thermal_output=df_thermal, 
    df_hp_electrical=df_hp_power, 
    df_pump_electrical=df_pump_power, 
    include_pump_power=True, 
    min_power_threshold=0.1, 
    logger=None
)


modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - Starting heat pump COP calculation
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - Processing data with shape: (35041, 32)
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - Include pump power: True
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - Minimum power threshold: 0.1
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - Using heat pump + pump electrical power for COP calculation
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - Calculating COP statistics
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO - COP calculation completed:
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO -   Total calculations: 1121312
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO -   Valid calculations: 267608
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO -   Success rate: 23.9%
modules.heat_pump_analyzer.calculate_heat_pump_cop - INFO -   Overall mean C

In [13]:
df_cop.head()

,T56_wa2b_efh2,T54_wa2b_efh1,T48_wa2b_rhh3_2,T50_wa2b_rhh3_3,T44_wa2b_dhh6_1,T42_wa2b_dhh6_2,T58_wa2b_dhh3_1,T36_wa2b_dhh3_2,T38_wa2b_dhh4_1,T40_wa2b_dhh4_2,...,T29_wa3_dhh2_2,T43_wa3_dhh5_1,T41_wa3_dhh5_2,T45_wa3_dhh6_1,T49_wa3_rhh2_2,T47_wa3_rhh2_3,T26_wa2b_rhh1_1,T24_wa2b_rhh1_2,T22_wa2b_rhh1_3,T18_wa2b_dhh1_1
2024-01-01 00:00:00,NaN,4.936554,NaN,NaN,NaN,4.939321,3.009283,NaN,3.009441,NaN,...,4.909973,NaN,4.90946,NaN,4.908976,4.909074,4.909966,NaN,4.910009,NaN
2024-01-01 00:15:00,NaN,NaN,5.01477,5.004477,NaN,4.679872,3.043729,NaN,3.042169,5.002821,...,NaN,NaN,NaN,4.989559,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:30:00,NaN,NaN,NaN,NaN,5.08313,NaN,3.088854,5.092157,3.075486,NaN,...,NaN,5.071995,NaN,NaN,NaN,NaN,NaN,5.153162,NaN,NaN
2024-01-01 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,3.127653,NaN,3.156219,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 01:00:00,5.340139,5.348206,NaN,NaN,NaN,NaN,3.140405,NaN,3.225374,NaN,...,NaN,NaN,NaN,NaN,5.219397,5.232491,NaN,NaN,NaN,5.408661


,T56_wa2b_efh2,T54_wa2b_efh1,T48_wa2b_rhh3_2,T50_wa2b_rhh3_3,T44_wa2b_dhh6_1,T42_wa2b_dhh6_2,T58_wa2b_dhh3_1,T36_wa2b_dhh3_2,T38_wa2b_dhh4_1,T40_wa2b_dhh4_2,...,T29_wa3_dhh2_2,T43_wa3_dhh5_1,T41_wa3_dhh5_2,T45_wa3_dhh6_1,T49_wa3_rhh2_2,T47_wa3_rhh2_3,T26_wa2b_rhh1_1,T24_wa2b_rhh1_2,T22_wa2b_rhh1_3,T18_wa2b_dhh1_1
2024-01-01 00:00:00,0.000000,716.109741,0.000000,0.000000,0.000000,311.352081,2379.914062,0.000000,2379.914062,0.00000,...,394.798462,0.000000,394.798462,0.000000,394.798462,394.798462,704.127686,0.000000,704.127686,0.000000
2024-01-01 00:15:00,0.000000,0.000000,386.314789,387.142242,0.000000,72.975403,2353.000000,0.000000,2354.243164,387.28595,...,0.000000,0.000000,0.000000,388.343994,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-01 00:30:00,0.000000,0.000000,0.000000,0.000000,380.987823,0.000000,2318.465332,380.285767,2328.614502,0.00000,...,0.000000,381.850006,0.000000,0.000000,0.000000,0.000000,0.000000,669.912781,0.000000,0.000000
2024-01-01 00:45:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2289.563232,0.000000,2268.742188,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-01 01:00:00,659.610474,658.582886,0.000000,0.000000,0.000000,0.000000,2280.005859,0.000000,2219.770752,0.00000,...,0.000000,0.000000,0.000000,0.000000,370.598877,369.648102,0.000000,0.000000,0.000000,357.278503


In [16]:
from typing import List

def save_dataframes_to_excel(
    dataframes: Dict[str, pd.DataFrame],
    file_path: str,
    index: bool = True,
    float_format: Optional[str] = None,
    sheet_order: Optional[List[str]] = None
) -> None:
    """
    Speichert mehrere DataFrames in einer Excel-Datei, wobei jeder DataFrame ein eigenes Sheet erhält.
    
    Parameters
    ----------
    dataframes : Dict[str, pd.DataFrame]
        Dictionary mit Sheet-Namen als Schlüssel und DataFrames als Werte
    file_path : str
        Pfad zur Excel-Datei
    index : bool, default True
        Ob der Index der DataFrames mitgespeichert werden soll
    float_format : Optional[str], default None
        Format für Fließkommazahlen (z.B. '%.2f' für 2 Dezimalstellen)
    sheet_order : Optional[List[str]], default None
        Liste mit Sheet-Namen in der gewünschten Reihenfolge
        
    Returns
    -------
    None
    
    Examples
    --------
    >>> df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
    >>> df2 = pd.DataFrame({'X': [10, 20], 'Y': [30, 40]})
    >>> save_dataframes_to_excel({'Sheet1': df1, 'Sheet2': df2}, 'output.xlsx')
    """
    # Verzeichnis erstellen, falls es nicht existiert
    os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)
    
    # ExcelWriter erstellen
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        # Falls eine bestimmte Reihenfolge gewünscht ist
        if sheet_order:
            # Sicherstellen, dass alle Sheets in sheet_order enthalten sind
            missing_sheets = set(dataframes.keys()) - set(sheet_order)
            if missing_sheets:
                sheet_order.extend(missing_sheets)
            
            # DataFrames in der angegebenen Reihenfolge speichern
            for sheet_name in sheet_order:
                if sheet_name in dataframes:
                    dataframes[sheet_name].to_excel(
                        writer, 
                        sheet_name=sheet_name, 
                        index=index,
                        float_format=float_format
                    )
        else:
            # DataFrames ohne bestimmte Reihenfolge speichern
            for sheet_name, df in dataframes.items():
                df.to_excel(
                    writer, 
                    sheet_name=sheet_name, 
                    index=index,
                    float_format=float_format
                )

save_dataframes_to_excel(
    {
        "COP": df_cop,
        "HP electric power": df_hp_power,
        "HP Q cond": df_thermal,
        "Pump Results": df_pump_power,
    },
    os.path.join(scenario["output_dir"], "HP_analysis.xlsx"),
    index=True,
    float_format="%.2f",
    sheet_order=["COP", "HP electric power", "HP Q cond", "Pump Results"]
)

In [ ]:

plot_monthly_energy_analysis(monthly_energy, save_path=os.path.join(scenario["output_dir"], "monthly_energy_analysis.png"))

In [72]:
import modules.heat_pump_analyzer as hp_analyzer
import importlib
importlib.reload(hp_analyzer)

<module 'modules.heat_pump_analyzer' from 'e:\\rka_lko\\git\\DHN-performance-toolkit\\modules\\heat_pump_analyzer.py'>

In [73]:
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'Helvetica'],
    'font.size': 16,              # Basic Font Size
    'axes.titlesize': 20,         # Title
    'axes.labelsize': 16,         # Acis-Labels
    'xtick.labelsize': 14,        # X-Ticks
    'ytick.labelsize': 14,        # Y-Ticks
    'legend.fontsize': 14,        # Legend
    'axes.titleweight': 'bold'
})

In [ ]:
plotter = hp_analyzer.HeatPumpPlotter(style='seaborn-v0_8-whitegrid', figsize_default=(12, 8))

fig1 = plotter.plot_energy_breakdown(
        df_monthly=monthly_energy,
        figsize=(14, 8),
        title='Customized Energy Breakdown',
        show_values=True,
        bar_width=0.8,
        legend_loc="upper center"
    )

In [ ]:
# 2. COP comparison plot
fig2 = plotter.plot_cop_comparison(
        monthly_energy,
        figsize=(12, 6),
        line_width=3,
        marker_size=10,
        show_seasonal_bg=True,
        legend_loc="upper center"
    )

In [ ]:
# 4. COP trend analysis
fig4 = plotter.plot_cop_trend_analysis(
        monthly_energy,
        figsize=(14, 8),
        show_annual_avg=True,
        show_seasonal_avg=True,
        legend_loc='upper center',
    )

In [62]:
import uesgraphs.analyze as analyze
uesgraph.graph["supply_type"] = "supply"
analyze.assign_data_to_uesgraphs(uesgraph,
                                scenario["data_path"],
                                start_date=DEFAULT_PARAMS["start_date"], 
                                end_date=DEFAULT_PARAMS["end_date"],
                                time_interval=DEFAULT_PARAMS["time_interval"])


Processing: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\Sim20250612_154050_1\Results\Sim20250612_154050_1_inputs.gzip
Assignment of pressure to nodes completed


<uesgraphs.UESGraph object>

In [31]:
#
#import modules.utils as ut
#ut.load_pipe_catalog(custom_path="data")

In [63]:

import modules.velocity as velocity
import importlib
importlib.reload(velocity)

analyzed_graph = velocity.velocity_analysis_pipeline(uesgraph,catalog=r"data\isoplus.csv")

In [ ]:

velocity.plot_velocity_vs_diameter_simple(graph=analyzed_graph,
                                      catalog=r"data\isoplus.csv",
                                      save_path=os.path.join(scenario["output_dir"], "velocity_diameter_plot.png"),
                                      max_velocity=2.5,
                                      velocity_metrics = ["max"]
                                      )

In [65]:
analyzed_graph.graph["name"] = "Hassel"
velocity.plot_network_velocity_analysis(
    analyzed_graph,
    save_path=os.path.join(scenario["output_dir"], "velocity_analysis_cum_min"),
    analysis_metric="cum_error",
    constraint_type="min"
)

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\2\Visuals_20250623_120108.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


In [ ]:


# Get list of edges with velocity data
edges_with_velocity = []
for edge in analyzed_graph.edges:
    if 'velocity' in analyzed_graph.edges[edge]:
        edges_with_velocity.append(edge)

print(f"Found {len(edges_with_velocity)} edges with velocity data")

# Plot time series for first few pipes (or specific ones)
num_pipes_to_plot = min(3, len(edges_with_velocity))  # Plot max 3 pipes

for i, edge in enumerate(edges_with_velocity[:num_pipes_to_plot]):
    edge_data = analyzed_graph.edges[edge]
    
    # Prepare pipe data dictionary for the plotting function
    pipe_data = {
        'velocity': edge_data['velocity'],
        'name': f"Edge_{edge[0]}_{edge[1]}",  # Edge name
        'constraints': edge_data.get('constraints', {}),
    }
    
        
    save_path = os.path.join(scenario["output_dir"], f"velocity_constraint_time_series_{i+1}.png")

    velocity.plot_velocity_time_series(
        pipe_data=pipe_data,
        key="velocity",
        num_points=500,  # Adjust for performance (35041 to plot whole year)
        save_path=save_path
    )



In [ ]:
velocity.plot_velocity_time_series(
    pipe_data=pipe_data,
    key="velocity",
    num_points=35041,  # Adjust for performance
)